<a href="https://colab.research.google.com/github/reidnersousa/IFB_Compiladores_2023/blob/main/analisador_sintatico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
class SyntaxAnalyzer:
    def __init__(self, lexer):
        self.lexer = lexer
        self.current_token = self.lexer.next_token()

    def eat(self, token_type):
        if self.current_token.type == token_type:
            self.current_token = self.lexer.next_token()
        else:
            raise SyntaxError(f'Expected {token_type}, got {self.current_token.type} instead.')

    def factor(self):
        token = self.current_token
        if token.type == 'LPAREN':
            self.eat('LPAREN')
            expr = self.expr()
            self.eat('RPAREN')
            return expr
       
        if token.type == 'INTEGER' or token.type == 'FLOAT':
            self.eat(token.type)
            return token.value


        elif token.type =='PONTO':
            self.eat('PONTO')
            return token.value
        else:
            raise SyntaxError(f'Unexpected token {token.type}')

    def term(self):
        result = self.factor()

        while self.current_token.type in ('MULT', 'DIV','POWER'):
            op = self.current_token
            if op.type == 'MULT':
                self.eat('MULT')
                result *= self.factor()
            elif op.type == 'DIV':
                self.eat('DIV')
                result /= self.factor()
            elif op.type =='POWER':
                self.eat('POWER')
                result **= self.factor()

        return result

    def expr(self):
        result = self.term()

        while self.current_token.type in ('PLUS', 'MINUS'):
            op = self.current_token
            if op.type == 'PLUS':
                self.eat('PLUS')
                result += self.term()
            elif op.type == 'MINUS':
                self.eat('MINUS')
                result -= self.term()

        return result

    def parse(self):
        print(self.current_token)
        print("lexer",self.lexer)
        result = self.expr()
        if self.current_token.type != 'EOF':
            raise SyntaxError('Esperar token no final  da expressão')
        return result




In [48]:
class Lexer:
  
  
  def __init__(self, text):
  
    self.text = text
    self.pos = 0

  def next_token(self):
    
    
    if self.pos >= len(self.text):
        return Token('EOF', None)
    current_char = self.text[self.pos]

    if current_char.isdigit():
        value = ''
        while self.pos < len(self.text) and self.text[self.pos].isdigit():
            value += self.text[self.pos]
            self.pos += 1
        return Token('INTEGER', int(value))
    
    if current_char.isdigit():
      value = ''
      while self.pos < len(self.text) and (self.text[self.pos].isdigit() or self.text[self.pos] == '.'):
          value += self.text[self.pos]
          self.pos += 1
      if value.count('.') > 1:
          raise ValueError(f"Invalid float value: {value}")
      if value[-1] == '.':
          raise ValueError(f"Invalid float value: {value}")
      if '.' in value:
          return Token('FLOAT', float(value))
      else:
          return Token('INTEGER', int(value))



    if current_char == '*':
      if self.pos < len(self.text)-1 and self.text[self.pos+1] == '*':
          self.pos += 2
          return Token('POWER', '**')
      else:
          self.pos += 1
          return Token('MULT', '*')


    if current_char == '+':
        self.pos += 1
        return Token('PLUS', '+')

    if current_char == '-':
        self.pos += 1
        return Token('MINUS', '-')

    

    if current_char == '/':
        self.pos += 1
        return Token('DIV', '/')

  

    if current_char == '(':
        self.pos += 1
        return Token('LPAREN', '(')

    if current_char == ')':
        self.pos += 1
        return Token('RPAREN', ')')

    if current_char == '.':
         value = ''
         self.pos += 1
         while self.pos < len(self.text) and self.text[self.pos].isdigit():
           
           value += self.text[self.pos]
           self.pos += 1
         return Token('FLOAT', float(f"{current_char}{value}"))


    # if none of the above match, raise an error
    raise ValueError(f"Invalid character: {current_char}")


In [44]:
class Token:
    def __init__(self, type_, value):
        self.type = type_
        self.value = value

    def __str__(self):
        return f'Token({self.type}, {self.value})'


In [55]:
lexer = Lexer('(12+1)+2*10')
analyzer = SyntaxAnalyzer(lexer)
result = analyzer.parse()
print(result)

soma=(12+1)+2*10
print(soma)
power =10
power**=2
print(power) # 24

Token(LPAREN, ()
lexer <__main__.Lexer object at 0x7ff4e7f43e50>
33
33
100


In [58]:
lexer = Lexer('2.0+4.0')
analyzer = SyntaxAnalyzer(lexer)
result = analyzer.parse()
print(result)

Token(INTEGER, 2)
lexer <__main__.Lexer object at 0x7ff4e7f416c0>


SyntaxError: ignored